## 0. Imports

In [555]:
import pandas as pd
import re

import torch
from torch import nn
from torchvision.transforms import ToTensor
from torchmetrics.classification import BinaryAccuracy
from tqdm.notebook import tqdm

## 1. Data preparation and cleaning

### Load dataset

In [513]:
df = pd.read_excel('movies.xlsx')
pd.set_option('display.max_columns', None)

# Truncate
pd.set_option('display.max_rows', False)

# Dont truncate
# pd.set_option('display.max_rows', None)

### Dataset info

In [514]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1402 entries, 0 to 1401
Data columns (total 31 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Film                                     1402 non-null   object 
 1   Year                                     1402 non-null   int64  
 2   Script Type                              1402 non-null   object 
 3   Rotten Tomatoes  critics                 1401 non-null   object 
 4   Metacritic  critics                      1402 non-null   object 
 5   Average critics                          1402 non-null   object 
 6   Rotten Tomatoes Audience                 1401 non-null   float64
 7   Metacritic Audience                      1402 non-null   object 
 8   Rotten Tomatoes vs Metacritic  deviance  1402 non-null   object 
 9   Average audience                         1402 non-null   object 
 10  Audience vs Critics deviance             1402 no

In [515]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,1402.0,2012.140514,3.190291,2007.0,2009.0000,2012.000,2015.0000,2017.00
Rotten Tomatoes Audience,1401.0,62.122769,17.435737,17.0,49.0000,62.000,76.0000,98.00
Opening weekend ($million),1402.0,23.200357,28.547541,0.0,6.6025,14.200,28.5975,247.97
Domestic gross ($million),1402.0,74.058024,87.389737,0.0,21.0900,44.045,92.5900,936.66
Worldwide Gross ($million),1402.0,173.008302,233.859025,0.0,38.0000,88.690,208.7500,2781.50
Distributor,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IMDb Rating,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IMDB vs RT disparity,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [516]:
df.describe(include='object').T

,count,unique,top,freq
Film,1402,1396,Big Miracle,2
Script Type,1402,17,original screenplay,546
Rotten Tomatoes critics,1401,102,93,29
Metacritic critics,1402,90,51,39
Average critics,1402,196,64,25
Metacritic Audience,1402,78,66,50
Rotten Tomatoes vs Metacritic deviance,1402,82,-1,58
Average audience,1402,116,58,45
Audience vs Critics deviance,1402,78,-5,54
...,...,...,...,...


### Drop empty, unecessary and extraneous columns for classification

In [517]:
df = df.drop(['Distributor', 'IMDb Rating', 'IMDB vs RT disparity', 'Opening Weekend', 'Domestic Gross', 'Foreign Gross', 'Worldwide Gross', 'Film', 'Year', 'Release Date (US)', 'Oscar Detail', 'Rotten Tomatoes vs Metacritic  deviance', 'Audience vs Critics deviance ', 'Worldwide Gross ($million)', ' of Gross earned abroad', 'Average audience ', 'Average critics ', ' Budget recovered', ' Budget recovered opening weekend'], axis=1)

### Handle missing values

In [518]:
missing_data = df.isnull().sum()
missing_percentage = (missing_data[missing_data > 0] / df.shape[0]) * 100

In [519]:
print('Missing data\n--------------')
print(missing_data)


Missing data
--------------
Script Type                      0
Rotten Tomatoes  critics         1
Metacritic  critics              0
Rotten Tomatoes Audience         1
Metacritic Audience              0
Primary Genre                 1383
Genre                            1
Opening weekend ($million)       0
Domestic gross ($million)        0
Foreign Gross ($million)         0
Budget ($million)                0
Oscar Winners                 1346
dtype: int64


In [520]:
print('Missing data percentages\n--------------')
print(missing_percentage)

Missing data percentages
--------------
Rotten Tomatoes  critics      0.071327
Rotten Tomatoes Audience      0.071327
Primary Genre                98.644793
Genre                         0.071327
Oscar Winners                96.005706
dtype: float64


In [521]:
df = df.dropna(subset=['Genre', 'Rotten Tomatoes  critics', 'Rotten Tomatoes Audience '])

In [522]:
# Drop invalid lines
df = df[~df.isin(['-']).any(axis=1)]
df[(df == '-').any(axis=1)]

,Script Type,Rotten Tomatoes critics,Metacritic critics,Rotten Tomatoes Audience,Metacritic Audience,Primary Genre,Genre,Opening weekend ($million),Domestic gross ($million),Foreign Gross ($million),Budget ($million),Oscar Winners


In [523]:
missing_data = df.isnull().sum()
missing_data

Script Type                      0
Rotten Tomatoes  critics         0
Metacritic  critics              0
Rotten Tomatoes Audience         0
Metacritic Audience              0
Primary Genre                 1372
Genre                            0
Opening weekend ($million)       0
Domestic gross ($million)        0
Foreign Gross ($million)         0
Budget ($million)                0
Oscar Winners                 1335
dtype: int64

### Check for dublicates

In [524]:
df.duplicated(keep=False).sum()

0

In [525]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Rotten Tomatoes Audience,1391.0,62.048167,17.428369,17.0,49.000,62.00,76.000,98.00
Opening weekend ($million),1391.0,23.327541,28.607251,0.0,6.755,14.30,28.720,247.97
Domestic gross ($million),1391.0,74.500093,87.569797,0.0,21.440,44.34,93.455,936.66


### Convert `script type` and `genre` to categorical values

In [526]:
df['Script Type'] = df['Script Type'].str.lower()

In [527]:
script_types = df['Script Type'].unique()

In [528]:
# Split strings and remove trailing spaces
types = []
for str in script_types:
    types.append(re.split(' / |, ',str))
script_types = [j.strip() for sub in types for j in sub]
#Convert to set to remove dublicates
script_types = set(script_types)

In [529]:
# Insert columns for each coresponding script type
for i, item in enumerate(script_types):
    df.insert(i+1, 'st_'+item, 0)

In [530]:
# Assign 1 to each corresponding script type
for i in df.index:
    for script in script_types:
        if script in df.at[i, 'Script Type']:
            df.at[i, 'st_'+script] = 1

In [531]:
df = df.drop(['Script Type'], axis=1)

In [532]:
df = df.drop(['Primary Genre'], axis=1)

In [533]:
# Convert genres to lower to avoid possible dublication
df['Genre'] = df['Genre'].str.lower()
genres = df['Genre'].unique()

In [534]:
# Split strings and remove trailing spaces
unique_genres = []
for str in genres:
    unique_genres.append(re.split(', |. ',str))
genre_types = [j.strip() for sub in unique_genres for j in sub]
#Convert to set to remove dublicates

genre_types = set(genre_types)

In [535]:
# Remove invalid values
genre_types.remove('')
genre_types.remove('adventur')
genre_types.remove('biograph')
genre_types.remove('comed')
genre_types.remove('crim')
genre_types.remove('documentar')
genre_types.remove('dram')
genre_types.remove('famiily')
genre_types.remove('famil')
genre_types.remove('fantas')
genre_types.remove('horro')
genre_types.remove('musi')
genre_types.remove('music')
genre_types.remove('myster')
genre_types.remove('romanti')
genre_types.remove('sci-f')
genre_types.remove('sport')
genre_types.remove('thriler')
genre_types.remove('thrille')

In [536]:
genre_types

{'action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'history',
 'horror',
 'musical',
 'mystery',
 'period',
 'romance',
 'sci-fi',
 'sports',
 'thriller',
 'war',
 'western'}

In [537]:
# Insert columns for each coresponding script type
for i, item in enumerate(genre_types):
    df.insert(i+8, 'gen_'+item, 0)

In [538]:
# Assign 1 to each corresponding script type
for i in df.index:
    for genre in genre_types:
        if genre in df.at[i, 'Genre']:
            df.at[i, 'gen_'+genre] = 1

In [539]:
df = df.drop(['Genre'], axis=1)

### Convert `Oscar Winners` to gategorical numerical values

In [540]:
df['Oscar Winners'] = df["Oscar Winners"].notnull().astype("int")

### Convert to floats

In [541]:
df = df.replace(',','', regex=True)
df = df.astype('float32')

## 2. Classification

### Prepare training and testing splits

In [542]:
features = df.drop(['Oscar Winners'], axis=1)
labels = df['Oscar Winners']

X = torch.from_numpy(features.to_numpy(dtype='float32'))
y = torch.from_numpy(labels.to_numpy(dtype='float32'))

In [543]:
X.shape, y.shape

(torch.Size([1391, 37]), torch.Size([1391]))

In [544]:
# Split at 30%
split_idx = int(X.shape[0] * 0.7)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx].unsqueeze(dim=1), y[split_idx:].unsqueeze(dim=1)

In [545]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([973, 37]),
 torch.Size([418, 37]),
 torch.Size([973, 1]),
 torch.Size([418, 1]))

In [546]:
X_train.dtype, X_test.dtype, y_train.dtype, y_test.dtype

(torch.float32, torch.float32, torch.float32, torch.float32)

### Define classification neural network

In [573]:
class OscarClassification(nn.Module):
    def __init__(self, input_features: int, output_features: int):
        super(OscarClassification, self).__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=20, dtype=torch.float32),
            # nn.ReLU(),
            # nn.Linear(in_features=20, out_features=20, dtype=torch.float32),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=output_features, dtype=torch.float32)
        )
    
    def forward(self, x: torch.Tensor):
        return self.layer_stack(x)


### Define loss, optimizer and accuracy functions and initialize model

In [574]:
# Calculate accuracy
def accuracy_fn(y_pred, y_true):
    # print(f'prediction {y_pred} true: {y_true} ')
    correct = torch.eq(y_true, y_pred).sum().item()
    return (correct / len(y_pred)) * 100

In [575]:
# Setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device used: {device}')

# Initialize model
model = OscarClassification(input_features=37, output_features=1).to(device)

# Accuracy function
# accuracy_fn = BinaryAccuracy().to(device) # metric(preds, target)

# Loss function
loss_fn = torch.nn.BCEWithLogitsLoss().to(device)

# Optimizer function
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer= torch.optim.Adam(model.parameters(), lr=0.01)


Device used: cuda


## Training loop

In [576]:
# torch.manual_seed(42)

epochs = 100

X_train, y_train, X_test, y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)

# start_time = timer()
# start_resources = psutil.cpu_times()

# for epoch in tqdm(range(epochs), desc='Training...'):
for epoch in range(epochs):
    model.train()

    # 1. Forward passs
    y_hat = model(X_train)
    loss = loss_fn(y_hat, y_train)    

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Calculate prediction accuracy
    y_train_pred = torch.round(torch.sigmoid(y_hat))        #if not using sigmoid on model output
    # y_train_pred = torch.round(y_hat)                     #prediction probabilities -> prediction labels
    acc = accuracy_fn(y_pred=y_train_pred, y_true=y_train)

    ### Testing
    model.eval()
    with torch.inference_mode():
        y_hat = model(X_test)
        test_loss = loss_fn(y_hat, y_test)

        y_test_pred = torch.round(torch.sigmoid(y_hat))
        # y_test_pred = torch.round(y_hat)
        test_acc = accuracy_fn(y_pred=y_test_pred, y_true=y_test)

    if epoch % 1 == 0 and epoch != 0:
        print(f'Epoch: {epoch:04d} | Loss: {loss:.4f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')

# end_time = timer()
# end_resources = start_resources = psutil.cpu_times()

# print(f'Resources: {start_resources}')


# print_train_time(start=start_time, end=end_time, device=device)

Epoch: 0001 | Loss: 4.0998, Acc: 95.27% | Test Loss: 0.2164, Test Acc: 96.17%
Epoch: 0002 | Loss: 0.9329, Acc: 95.89% | Test Loss: 0.4848, Test Acc: 88.28%
Epoch: 0003 | Loss: 15.2754, Acc: 89.00% | Test Loss: 0.2866, Test Acc: 94.74%
Epoch: 0004 | Loss: 8.2008, Acc: 94.96% | Test Loss: 0.3483, Test Acc: 96.17%
Epoch: 0005 | Loss: 0.8366, Acc: 95.89% | Test Loss: 0.4025, Test Acc: 96.17%
Epoch: 0006 | Loss: 2.1895, Acc: 95.89% | Test Loss: 0.4115, Test Acc: 96.17%
Epoch: 0007 | Loss: 3.1553, Acc: 95.89% | Test Loss: 0.3899, Test Acc: 96.17%
Epoch: 0008 | Loss: 3.8102, Acc: 95.89% | Test Loss: 0.3539, Test Acc: 96.17%
Epoch: 0009 | Loss: 4.2110, Acc: 95.89% | Test Loss: 0.3142, Test Acc: 96.17%
Epoch: 0010 | Loss: 4.4043, Acc: 95.89% | Test Loss: 0.2800, Test Acc: 96.17%
Epoch: 0011 | Loss: 4.4261, Acc: 95.89% | Test Loss: 0.2477, Test Acc: 96.17%
Epoch: 0012 | Loss: 4.3045, Acc: 95.89% | Test Loss: 0.2205, Test Acc: 96.17%
Epoch: 0013 | Loss: 4.0620, Acc: 95.89% | Test Loss: 0.2001, Te

### Test on test split

In [577]:
with torch.inference_mode():
    X_test, y_test = X_test.to(device), y_test.to(device)
    y_test_logits = model(X_test)
    y_test_pred = torch.round(torch.sigmoid(y_test_logits))
print(f'Test accuracy: {accuracy_fn(y_pred=y_test_pred, y_true=y_test):.2f}%')

Test accuracy: 95.93%


In [582]:
correct_oscar_guesses = 0
total_oscars = 0
for pred, truth in zip(y_test_logits, y_test):   
    if torch.round(torch.sigmoid(pred)) == 1 and truth == 1:
        correct_oscar_guesses += 1
    if truth == 1:
        total_oscars += 1

print(f'Correct oscar guesses: {correct_oscar_guesses} out of {total_oscars}')

Correct oscar guesses: 0 out of 16


### Problem

Model wont predict any oscars because oscar winning movies are very few compared to non oscar winning ones